<a href="https://colab.research.google.com/github/Lou1108/DeepLearning/blob/main/Assignment2/meg_prediction_sofia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What we need to do
*   do the pre processing seperately and save the data
*   check and try different models to see if there's one working better with the data
* grid search on the number of the neurons in the layers



### Assignment 2

# Imports and Variables


In [1]:
import os
import glob
import h5py
import numpy as np
from collections import Counter

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.signal import butter, filtfilt, decimate

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    Conv2D,
    DepthwiseConv2D,
    SeparableConv2D,
    MaxPooling1D,
    AveragePooling2D,
    LSTM,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    Activation,
)


2025-05-28 16:54:35.919083: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BASE_PATH = "Final Project data/Intra"
TRAIN_PATH = os.path.join(BASE_PATH, 'train/')
TEST_PATH = os.path.join(BASE_PATH, 'test/')


# data specific
NUM_CHANNELS = 248
NUM_CLASSES = 4
LABEL_MAP = {'rest':0, 'task_motor':1, 'task_story_math':2, 'task_working_memory':3}
NUM_CLASSES = len(LABEL_MAP)
orig_fs=2034
target_fs=250
DOWNSAMPLE_FACTOR =  int(orig_fs / target_fs)

# Model specific
NUM_EPOCHS = 1
NORMALIZATION_METHOD = "time"  # Choose: "minmax", "zscore", or "perchannel"

In [3]:
def load_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        # Extractin the label
        filename = file_path.split('/')[-1]

        #handling the different task naming conventions
        if 'rest' in filename:
            labels.append(LABEL_MAP['rest'])
        elif 'motor' in filename:
            labels.append(LABEL_MAP['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(LABEL_MAP['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(LABEL_MAP['task_working_memory'])
        else:
            # iff a file doesn't match
            print(f"Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            # Instead of guessing the dataset name, we get the first key from the file
            # This is robust because we know there is only one dataset per file[cite: 10].
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    #convert to numpy arrays
    return np.array(data), np.array(labels)

In [4]:
train_files = glob.glob(f"{TRAIN_PATH}/*.h5")
test_files = glob.glob(f"{TEST_PATH}/*.h5")

X_train, y_train = load_data(train_files)
X_test, y_test = load_data(test_files)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Uniqe labels: {np.unique(y_train)}")
print(f"Number of training samples: {len(X_train)}")

Shape of X_train: (32, 248, 35624)
Shape of y_train: (32,)
Shape of X_test: (8, 248, 35624)
Uniqe labels: [0 1 2 3]
Number of training samples: 32


# Load and Preprocess Data
Apply a lowpass filter for downsampling the frequency

In [5]:
# lowpass filter ---> check it
# def bandpass_filter(data, lowcut=1.0, highcut=150.0, fs=2034, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = butter(order, [low, high], btype='band')
#     return filtfilt(b, a, data, axis=-1)

Normalization functions

In [6]:
def min_max_scale_sample(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data.reshape(-1, 1)).reshape(data.shape)

def z_score_normalize(data):
    mean = data.mean(axis=-1, keepdims=True)
    std = data.std(axis=-1, keepdims=True)
    return (data - mean) / (std + 1e-8)

def time_norm(data):
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

In [7]:
def normalization(data):
    if NORMALIZATION_METHOD == "minmax":
        data = min_max_scale_sample(data)
    elif NORMALIZATION_METHOD == "zscore":
        data = z_score_normalize(data)
    elif NORMALIZATION_METHOD == "time":
        data = time_norm(data)
    return data

In [8]:
def normalize_data(data):
    # Data shape is (n_samples, n_channels, n_timesteps)
    # We want to scale each of the (n_samples * n_channels) time series

    # Reshape to (n_samples * n_channels, n_timesteps) to apply StandardScaler
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

In [9]:
X_train.shape

(32, 248, 35624)

In [10]:
N_TIMESTEPS = X_train.shape[2]
X_train_ds=decimate(X_train, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
X_test_ds=decimate(X_test, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
X_train_ds = X_train[:, :, ::DOWNSAMPLE_FACTOR]
X_test_ds = X_test[:, :, ::DOWNSAMPLE_FACTOR]

N_TIMESTEPS_DS = X_train_ds.shape[2]

print(f"Original number of time steps: {N_TIMESTEPS}")
print(f"Downsampled number of time steps: {N_TIMESTEPS_DS}")

Original number of time steps: 35624
Downsampled number of time steps: 4453


In [11]:
X_train_norm = normalization(X_train_ds)
X_test_norm = normalization(X_test_ds)

In [12]:
#DL models in Keras often expect the channel dimension last
#reshaping from (samples, channels, timesteps) to (samples, timesteps, channels)
X_train_final = np.transpose(X_train_norm, (0, 2, 1))
X_test_final = np.transpose(X_test_norm, (0, 2, 1))

print(f"Final shape of training data for the model: {X_train_final.shape}")

Final shape of training data for the model: (32, 4453, 248)


# Define Models

###CNN model

In [13]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        #1st convolutional block
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        #2nd convolutional block
        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # 3rd convolutional block
        Conv1D(filters=256, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # Flatten the features and feed to dense layers
        Flatten(),

        # dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

In [14]:
INPUT_SHAPE = (N_TIMESTEPS_DS, NUM_CHANNELS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 4453, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4453, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1113, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1113, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1113, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 278, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 278, 256)       │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 278, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 69, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 17664)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,261,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,832,196 (10.80 MB)

 Trainable params: 2,831,300 (10.80 MB)

 Non-trainable params: 896 (3.50 KB)

Training and evaluation of CNN

In [15]:
history_cnn = cnn_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,      # smaller batch size for better generalization
    validation_split=0.2 # 20% of training data for validation
)

# Evaluate on test Sets
loss_cnn, acc_cnn = cnn_model.evaluate(X_test_final, y_test, verbose=0)
print(f"accuracy on test set: {acc_cnn * 100:.2f}%")

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 897ms/step - accuracy: 0.3825 - loss: 2.6044 - val_accuracy: 0.1429 - val_loss: 1.4193
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 805ms/step - accuracy: 0.8992 - loss: 0.2740 - val_accuracy: 0.4286 - val_loss: 1.9656
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step - accuracy: 1.0000 - loss: 2.6577e-05 - val_accuracy: 0.2857 - val_loss: 3.8542
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step - accuracy: 0.9733 - loss: 0.1348 - val_accuracy: 0.2857 - val_loss: 5.0449
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step - accuracy: 0.9467 - loss: 0.6226 - val_accuracy: 0.4286 - val_loss: 5.5432
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 703ms/step - accuracy: 0.9200 - loss: 3.6659 - val_accuracy: 0.2857 - val_loss: 12.8350
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.2857 - val_loss: 20.4276
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 624ms/step - accuracy: 0.9733 - loss: 0.5022 - val_accuracy: 0.2857 - va

Training and evaluation LSTM Model

In [16]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # Convolutional layers to extract features
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # LSTM layer to model temporal sequences of the extracted features
        LSTM(128, return_sequences=False), # return_sequences=False  it's the last recurrent layer

        # Dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model


In [17]:
INPUT_SHAPE = (N_TIMESTEPS_DS, NUM_CHANNELS)

cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 4453, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 4453, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 1113, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 1113, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1113, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 278, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 390,212 (1.49 MB)

 Trainable params: 389,828 (1.49 MB)

 Non-trainable params: 384 (1.50 KB)

In [18]:
#CNN-LSTM Training and Evaluation
print("Starting CNN-LSTM model training...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2
)

# Evaluate on test sets
loss_hybrid, acc_hybrid = cnn_lstm_model.evaluate(X_test_final, y_test, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 1: {acc_hybrid * 100:.2f}%")

Starting CNN-LSTM model training...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3175 - loss: 1.4691 - val_accuracy: 0.4286 - val_loss: 1.3082
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 666ms/step - accuracy: 0.7358 - loss: 0.9642 - val_accuracy: 0.4286 - val_loss: 1.3155
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 752ms/step - accuracy: 0.9050 - loss: 0.5176 - val_accuracy: 0.4286 - val_loss: 1.2767
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 719ms/step - accuracy: 1.0000 - loss: 0.3580 - val_accuracy: 0.4286 - val_loss: 1.1987
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 684ms/step - accuracy: 1.0000 - loss: 0.1567 - val_accuracy: 0.4286 - val_loss: 1.0808
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 1.0000 - loss: 0.1481 - val_accuracy: 0.7143 - val_loss: 0.9710
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 793ms/step - accuracy: 1.0000 - loss: 0.0843 - val_accuracy: 0.7143 - val_loss: 0.8655
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 823ms/step - accuracy: 0.9525 - loss: 0.0765 - va

Training EEGNet model

In [19]:
def build_eegnet_model(num_classes, channels, timesteps, dropout_rate=0.5):
    input_layer = Input(shape=(channels, timesteps, 1))

    # temporal convolution
    block1 = Conv2D(16, (1, 64), padding='same', use_bias=False)(input_layer)
    block1 = BatchNormalization()(block1)

    #Depthwise spatial convolution
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=tf.keras.constraints.max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropout_rate)(block1)

    # separable convolution
    block2 = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropout_rate)(block2)

    # classification head
    flatten_layer = Flatten()(block2)
    dense_layer = Dense(num_classes, kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten_layer)
    output_layer = Activation('softmax')(dense_layer)

    return Model(inputs=input_layer, outputs=output_layer)

In [22]:
# Reshape data
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test_eegnet = X_test_norm[..., np.newaxis]
print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=NUM_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

# Define the earlystopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

Shape of data for EEGNet: (32, 248, 4453, 1)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 248, 4453, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 248, 4453, 16)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 248, 4453, 16)  │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 4453, 32)    │         7,936 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1, 4453, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 4453, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 1, 1113, 32)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1113, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 1, 1113, 32)    │         1,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1, 1113, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1, 1113, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 1, 139, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1, 139, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4448)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │        17,796 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,612 (111.77 KB)

 Trainable params: 28,452 (111.14 KB)

 Non-trainable params: 160 (640.00 B)

In [23]:
history_eegnet = eegnet_model.fit(
    X_train_eegnet,
    y_train,
    epochs=20,  # We can still set a high number, but early stopping will likely stop it earlier
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

print("\nEvaluating EEGNet model on test sets")

loss_eegnet, acc_eegnet = eegnet_model.evaluate(X_test_eegnet, y_test, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 1: {acc_eegnet * 100:.2f}%")

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 125s 26s/step - accuracy: 0.2492 - loss: 1.8498 - val_accuracy: 0.1429 - val_loss: 1.3777
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 97s 26s/step - accuracy: 0.7775 - loss: 0.6824 - val_accuracy: 0.4286 - val_loss: 1.3512
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 55s 19s/step - accuracy: 0.9733 - loss: 0.2233 - val_accuracy: 0.4286 - val_loss: 1.3133
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 69s 18s/step - accuracy: 0.8725 - loss: 0.2264 - val_accuracy: 0.4286 - val_loss: 1.2702
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 54s 15s/step - accuracy: 1.0000 - loss: 0.0572 - val_accuracy: 0.4286 - val_loss: 1.2206
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 62s 17s/step - accuracy: 1.0000 - loss: 0.0353 - val_accuracy: 0.5714 - val_loss: 1.1727
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 69s 18s/step - accuracy: 1.0000 - loss: 0.0273 - val_accuracy: 0.7143 - val_loss: 1.1284
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 62s 17s/step - accuracy: 1.0000 - loss: 0.0139 - val_accuracy: 1.0000 - val_loss: 1.0875